In [1]:
input = """47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47"""

input = open("inputs/5").read()


def parse_input(input):
    dependencies, orders = input.split("\n\n")

    dependencies = dependencies.split("\n")
    orders = orders.split("\n")

    dependencies = [tuple(map(int, dep.split("|"))) for dep in dependencies]
    orders = [list(map(int, order.split(","))) for order in orders]

    return dependencies, orders

In [2]:
dependencies, orders = parse_input(input)

In [3]:
from collections import defaultdict


def make_graph(dependencies):
    graph = defaultdict(list)

    for d1, d2 in dependencies:
        graph[d1].append(d2)

    return graph

In [4]:
from itertools import chain


def topo_sort(graph):
    # NOTE: no cycle detection!
    order = []
    all_nodes = set(graph.keys()) | set(chain(*graph.values()))
    unvisited = all_nodes

    def dfs(node):
        if node in unvisited:
            unvisited.remove(node)

        for neighbor in graph[node]:
            if neighbor in unvisited:
                dfs(neighbor)

        # we've explored all of node's dependencies: safe to append
        order.append(node)

    while unvisited:
        start_node = unvisited.pop()
        dfs(start_node)

    return order

In [5]:
p1_s = 0
p2_s = 0

for order in orders:
    # we avoid the cycle issue by subgraphing for the order first
    subgraph = make_graph([d for d in dependencies if d[0] in order and d[1] in order])
    middle_idx = len(order) // 2
    topo_order = topo_sort(subgraph)
    topo_idxs = [topo_order.index(n) for n in order]
    sorted_topo_idxs = tuple(sorted(topo_idxs, reverse=True))

    middle_num = topo_order[sorted_topo_idxs[middle_idx]]
    if sorted_topo_idxs == tuple(topo_idxs):
        p1_s += middle_num
    else:
        p2_s += middle_num

p1_s, p2_s

(5732, 4716)